In [1]:
%load_ext autoreload
%autoreload 2

import os
import fitz
import boto3
from tqdm import tqdm

base_dir = os.getcwd()

"""Notebook to retrieve and save all text for all files on the S3 bucket as .txt files"""

'Notebook to retrieve and save all text for all files on the S3 bucket as .txt files'

In [2]:
s3_input = "asset/asset_files_new_ocr/"
s3_aws_profile = "assets"
s3_session = boto3.Session(profile_name=s3_aws_profile)
s3 = s3_session.resource('s3')
bucket = s3.Bucket('swissgeol-assets-swisstopo')

In [ ]:
objs = list(bucket.objects.filter(Prefix=s3_input))

In [4]:
pdf_path = "data/NAB/Berichte_NAB 10-025_Kurzarbeitsprogram Geothermiebohrung Schlattingen.PDF"

output_dir = "data/text_files/"

def key_to_filename(key):
    return key.split("/")[-1]

In [ ]:
with tqdm(total=len(objs)) as pbar:

    for obj in objs:
        pbar.set_postfix_str(obj.key)
        
        filename = key_to_filename(obj.key)
        if filename.endswith(".pdf"):
                    tmp_file_path = os.path.join("data", filename)
                    bucket.download_file(obj.key, tmp_file_path)

                    doc = fitz.open(tmp_file_path)
                    output_path = os.path.join(output_dir, filename + ".txt")
                    
                    with open(output_path, "w", encoding = "utf-8") as text_file:
                        for _,page in enumerate(doc):
                            text= page.get_text()
                            text_file.write(text)

                    os.remove(tmp_file_path)   

        
        pbar.update(1)

